In [1]:
%pip install opencage pandas plotly requests pandas #install libraries


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#import libraries here:
import requests
import pandas as pd

In [3]:
url = 'https://api.open-meteo.com/v1/forecast' # api url which im using a get request

def get_forecast(lat, long, forecast_d=5): #creating an api call for user's input
    params = {
        'latitude': lat,  
        'longitude': long, #have to convert city to lat,long
        'forecast_days': forecast_d, #number of days in forecast
        'temperature_unit': 'fahrenheit', # Change unit maybe later according to user pref
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'hourly': 'temperature_2m,precipitation',
        'timezone': 'America/New_York',  # Change according to your timezone - later user could maybe do this 
    }
    response = requests.get(url, params=params)
    return response.json()

In [4]:
from opencage.geocoder import OpenCageGeocode
key = '4e0e9443673d4093ab7e797721508ea3'  # my api key
geocoder = OpenCageGeocode(key) #gets lat,long from city

def get_cord(city):
    response =  geocoder.geocode(city)
    if  response:
        if len(response):
            return response[0]['geometry']['lat'], response[0]['geometry']['lng']
    else:
        print('city is not found')
        return None, None

In [5]:
city = input("Enter the city: ")  # user input
lat, long = get_cord(city)

if lat and long:
    forecast_data = get_forecast(lat, long)
    forecast_df = pd.DataFrame(forecast_data['hourly'])

In [6]:
forecast_df #print out 5 day forecast

,time,temperature_2m,precipitation
0,2023-06-07T00:00,62.5,0.0
1,2023-06-07T01:00,60.5,0.0
2,2023-06-07T02:00,58.4,0.0
3,2023-06-07T03:00,57.5,0.0
4,2023-06-07T04:00,56.0,0.0
...,...,...,...
115,2023-06-11T19:00,72.5,0.0
116,2023-06-11T20:00,72.0,0.0
117,2023-06-11T21:00,71.3,0.0
118,2023-06-11T22:00,70.0,0.0
